<a href="https://colab.research.google.com/github/Mohamed-Taha-Essa/Generative-AI/blob/main/audio_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With the aid of the following supporting code, perform zero-shot audio classification task using huggingface `pipeline`, to predict if the audio sample is one of the following:

["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane"]

- you may need to search how to preprocess audio samples and prepare it for the mode. *Hint:* making the data sampling_rate matching the model sampling_rate

In [1]:
!pip install transformers
!pip install datasets

In [2]:
from datasets import load_dataset

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50",split="train[0:20]")
audio_sample = dataset[15]
audio_sample

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


{'filename': '1-110537-A-22.wav',
 'fold': 1,
 'target': 22,
 'category': 'clapping',
 'esc10': False,
 'src_file': 110537,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.25811768, -0.26950073, -0.21060181, ..., -0.52600098,
         -0.65093994, -0.71273804]),
  'sampling_rate': 44100}}

In [3]:
#You may need to run this in colab if the Ipython.display is not working properly in your local env
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample['audio']['array'],
             rate=audio_sample['audio']["sampling_rate"])

In [4]:
#TODO
from transformers import pipeline
classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")


config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
pip install librosa

In [6]:
import librosa

# Check the original sampling rate
original_sr = audio_sample['audio']['sampling_rate']
print(f"Original sampling rate: {original_sr} Hz")

# Assume your model needs 16000 Hz (common for speech models)
target_sr = 16000

# Resample if needed
if original_sr != target_sr:
    audio_data = audio_sample['audio']['array']
    resampled_audio = librosa.resample(audio_data, orig_sr=original_sr, target_sr=target_sr)
    print(f"Resampled to {target_sr} Hz")
else:
    resampled_audio = audio_sample['audio']['array']
    print("Sampling rate already matches the target.")

Original sampling rate: 44100 Hz
Resampled to 16000 Hz


In [7]:
candidate_labels = ["Sound of a child crying", "Sound of vacuum cleaner", "Sound of a bird singing", "Sound of an airplane"]
prediction = classifier(resampled_audio, candidate_labels=candidate_labels, sampling_rate=16000)  # Pass candidate_labels and sampling_rate as keyword arguments

In [8]:
prediction

[{'score': 0.4569459855556488, 'label': 'Sound of vacuum cleaner'},
 {'score': 0.3931463062763214, 'label': 'Sound of a child crying'},
 {'score': 0.09486962109804153, 'label': 'Sound of an airplane'},
 {'score': 0.055038101971149445, 'label': 'Sound of a bird singing'}]

Make Fine Tuning for Data